In [1]:
# Install NLTK
!pip install nltk

In [2]:
# Download the Brown Corpus
import nltk
nltk.download('brown')

# Imports
from typing import List, Tuple

[nltk_data] Downloading package brown to /Users/natulko/nltk_data...
[nltk_data]   Package brown is already up-to-date!


# Natural Language Processing Assignment 1: The Notebook

This is the notebook for the first hand-in assignment for Natural Language Processing. The notebook counts for 50% of the total assignment, which counts towards 15% of the final grade. 

## Assignment 5 (20 points)

In the exercises you had the opportunity to practice with fitting $n$-gram language models on the Brown corpus. In this assignment you will fit an interpolated model and learn how to avoid zero counts by exploiting a heldout set.

In [3]:
# Imports + code from exercises
from nltk.corpus import brown
import random
from nltk.lm import MLE, Laplace
from nltk.lm.api import LanguageModel
from nltk.lm.preprocessing import padded_everygram_pipeline

def split_data(data, train_ratio: float, heldout_ratio: float):
    random.Random(42).shuffle(data)
    train_cutoff = int(train_ratio*len(data))
    heldout_cutoff = int((train_ratio+heldout_ratio)*len(data))
    return {'train': data[:train_cutoff], 'heldout': data[train_cutoff:heldout_cutoff], 'test': data[heldout_cutoff:]}

# data: a list of (unpadded) sentences that are split into words
# n: the maximum order of n-grams to include
# model_type: parameter to support different types of language models
def fit_language_model(data: List[List[str]], n: int, model_type=MLE):
    train, vocab = padded_everygram_pipeline(n, data) 
    model = model_type(n)
    model.fit(train, vocab)
    return model

def evaluate_language_model(model: LanguageModel, test_data: List[str], n: int):
    # Note that `padded_everygram_pipeline' adds n-1 SOS and EOS symbols
    # which trickles through in calculating perplexity. 
    # For example, padding for n=3 will add the bigram for (<s>, <s>) which is NOT present in the model for n=2.
    gram_data, _ = padded_everygram_pipeline(n, test_data)
    test_grams = [x for t in gram_data for x in model.vocab.lookup(t) if len(x)==n]
    return model.perplexity(test_grams)

# Prepare data
non_fiction = brown.sents(categories=['editorial', 'government', 'learned', 'news', 'reviews'])
non_fiction = [s for s in non_fiction if len(s)>=3]

fiction = brown.sents(categories=['adventure', 'fiction', 'lore', 'mystery', 'romance', 'science_fiction'])
fiction = [s for s in fiction if len(s)>=3]

fiction_data = split_data(fiction, 0.9, 0.05)
non_fiction_data = split_data(non_fiction, 0.9, 0.05)

### Part 5.1: training a language model (10 points)

You may recall from the exercises that a simple smoothed language model doesn't perform so well on a test set, and generates strange examples. In this assignment you will investigate the use of an interpolated language model, combining it with parameters extracted from a heldout set.

Given are the following data: like in the exercises, we work with the Brown corpus and specifically the fiction and non-fiction texts of the corpus. Moreover, we will fix all data to contain all $n$-grams up to $n=3$.

As a first step, make sure you have fiction and non-fiction models available with Laplace smoothing (you may have fit these in the exercises already). Then, fit similar models but now use the `AbsoluteDiscountingInterpolated` model type, that implements absolute discounting. Calculate the perplexity to realize that once again there will be zero counts because the model is unsmoothed.

In [4]:
# Part 5.1 SOLUTION:
from nltk.lm import AbsoluteDiscountingInterpolated

# make sure you have fiction and non-fiction models available with Laplace smoothing
fiction_model = fit_language_model(fiction_data['train'], 2, Laplace)
non_fiction_model = fit_language_model(non_fiction_data['train'], 2, Laplace)

# fit similar models with AbsoluteDiscountingInterpolated
fiction_model_adi = fit_language_model(fiction_data['train'], 2, AbsoluteDiscountingInterpolated)
non_fiction_model_adi = fit_language_model(non_fiction_data['train'], 2, AbsoluteDiscountingInterpolated)

# calculate perplexity
fiction_perplexity = evaluate_language_model(fiction_model, fiction_data['test'], 2)
non_fiction_perplexity = evaluate_language_model(non_fiction_model, non_fiction_data['test'], 2)

fiction_perplexity_adi = evaluate_language_model(fiction_model_adi, fiction_data['test'], 2)
non_fiction_perplexity_adi = evaluate_language_model(non_fiction_model_adi, non_fiction_data['test'], 2)

print(f'Fiction model perplexity: {fiction_perplexity}')
print(f'Non-fiction model perplexity: {non_fiction_perplexity}')
print(f'Fiction model (ADI) perplexity: {fiction_perplexity_adi}')
print(f'Non-fiction model (ADI) perplexity: {non_fiction_perplexity_adi}')


Fiction model perplexity: 2808.092983899774
Non-fiction model perplexity: 4488.435817363989
Fiction model (ADI) perplexity: inf
Non-fiction model (ADI) perplexity: inf


As we can see, indeed without Laplace's smoothing we get infinite perplexity, as ... # TODO

### Part 5.2: avoiding zero counts (10 points)

One way of avoiding zero counts is to make sure that unseen words are mapped to a special token `<UNK>` (unknown). But how do we quantify the $n$-gram probability of unknown words? Well, we can use the held out set to estimate these counts. The implementation of language models in NLTK then allows us to simply *update* a language model with more data.

Hence, your assignment here is to implement the method `update_model` below. It takes three inputs: a language model, the heldout data, and the parameter `n` for the maximum $n$-grams to include. What the method should do:

 1. Use the language model's vocabulary to map unseen words of the heldout data to the `<UNK>` token,
 2. Generate the $n$-grams for the (tokenized) heldout data,
 3. Update the language model with these counts

After successful implementation of the method, use it to update the interpolated model and calculate its perplexity. How does it compare to the perplexity of the Laplace smoothed model?

*Hint: you may take inspiration from the method `evaluate_language_model` above.*

In [17]:
# Part 5.2 SOLUTION:

def update_model(model: LanguageModel, heldout_data: List[List[str]], n: int):
    heldout_data = [[word if word in model.vocab else '<UNK>' for word in sent] for sent in heldout_data]
    heldout_grams, _ = padded_everygram_pipeline(n, heldout_data)
    model.counts.update(heldout_grams)
    return model

# update interpolated model
fiction_model_adi = update_model(fiction_model_adi, fiction_data['heldout'], 2)
non_fiction_model_adi = update_model(non_fiction_model_adi, non_fiction_data['heldout'], 2)

# calculate perplexity
fiction_perplexity_adi = evaluate_language_model(fiction_model_adi, fiction_data['test'], 2)
non_fiction_perplexity_adi = evaluate_language_model(non_fiction_model_adi, non_fiction_data['test'], 2)

print(f'Fiction model (ADI) perplexity after update: {fiction_perplexity_adi}')
print(f'Non-fiction model (ADI) perplexity after update: {non_fiction_perplexity_adi}')

Fiction model (ADI) perplexity after update: 237.51032877960017
Non-fiction model (ADI) perplexity after update: 311.7907156649726


## Assignment 6: part-of-speech tagging (20 points)

This assignment consists of two parts in which you will consider parts-of-speech (POS) and POS tagging.

### Part 6.1: Ambiguity in open and closed class words (10 points)

In the lecture and in the exercises you saw that a small portion of words is ambiguous, but they nevertheless cover a large portion of text corpora. You also learnt about the distinction between open (i.e. dynamic words that may appear and disappear over time) and closed class words (i.e. pretty much fixed sets of words).

Below are two lists: one corresponds to closed class words, and another corresponds to open class words. We wish to inspect the ambiguity of words that are open class and words that are open class, and figure out whether words can belong to both!

Your task is as follows:
1. Determine the percentage of *unique* words that occur both as open class and as closed class words. I.e. out of all *unique* words, how many of them have been tagged at least once as open and and at least once as closed class?
2. For the *unique* words that only occur as either open class words or closed class words, calculate the percentage of *unique* words in each class that is may receive multiple POS tags (in their respective class). Do you find that open class words are more ambiguous, or less ambiguous then closed class words?

*Hint: If you run into any performance issues because of iterating over large lists, remember that you can convert a list into a set first and then membership lookup in Python becomes linear.*

In [ ]:
from nltk.corpus import treebank

all_word_tags = [word_tag for tagged_sentence in treebank.tagged_sents() for word_tag in tagged_sentence]
closed_class_tags = [',', '$', '.', 'CD', 'CC', 'RP', "''", '``', 'IN', 'TO',':', 'DT', 'WP', 'PRP',
                     'SYM', 'EX', 'PRP$', 'POS', 'MD']
open_class_tags = ['JJ', 'JJR', 'JJS', 'VBG', 'VBN', 'VB', 'NNP', 'VBD', 'NN', 'NNS',  'VBZ', 'VBP', 'NNPS']
other_tags = ['RBR', '-LRB-', 'PDT', 'WRB', 'WDT', 'RB', 'LS', 'WP$',  'UH', 
              'FW', '-RRB-',  'RBS', '#', '-NONE-']

In [ ]:
# 6.1 SOLUTION:


### Part 6.2: Smoothing a tagger (10 points)

In the exercises you trained an HMM tagger that used `LidstoneProbDist` as an estimator for its probability distributions. This estimator generally applies some form of smoothing for its probability calculation. In general this would mean that the probability of a transition from one tag to the other is defined as:

$$ P_{\beta}(t_i | t_{i-1}) = \frac{C(t_{i-1} t_i) + \beta}{C(t_{i-1}) + \beta \cdot V} $$
        
where $V$ is the number of possible tags. A similar smoothing applies to the calculation of emission probabilities $P(w_i | t_i)$.

In the exercises the smoothing parameter was set as $\beta=0.1$.

In this assignment, you will try to find a better parameter on a given dataset. The code below loads in a portion of the Penn Treebank and has code for training and testing a Hidden Markov Model tagger.

Your task is the following:

1. Train a tagger with smoothing parameter $0.1$ and verify that its accuracy is $0.917$,
2. Train an unsmoothed tagger, i.e. the smoothing parameter should be set to $0$, and report its accuracy,
3. Now find a smoothing value for which the corresponding model reaches an accuracy greater than $0.917$. Is it higher or lower than $0.1$?

In [ ]:
from nltk import LidstoneProbDist
from nltk.tag.hmm import load_pos, HiddenMarkovModelTrainer, HiddenMarkovModelTagger
from nltk.lm import MLE, Laplace

from tqdm import tqdm

def train_hmm_tagger(train_data: List[List[Tuple[str, str]]],
                     estimator=lambda fd, bins: LidstoneProbDist(fd, 0.1, bins)) -> HiddenMarkovModelTagger:
    tags = list(set([t for s in train_data for w,t in s]))
    words = list(set([w for s in train_data for w,t in s]))
    trainer = HiddenMarkovModelTrainer(tags, words)
    hmm = trainer.train_supervised(train_data, estimator=estimator)
    return hmm

def test_hmm_tagger(tagger: HiddenMarkovModelTagger, test_data: List[List[Tuple[str, str]]]) -> List[List[Tuple[str, str]]]:
    return [tagger.tag([w for w,t in s]) for s in tqdm(test, position=0)]

def accuracy(trues: List[str], predictions: List[str]):
    return sum([t == p for t, p in zip(trues, predictions)])/len(trues)

In [ ]:
from nltk.corpus import treebank

import random

def split_data(data, cutoff=0.98):
    random.Random(42).shuffle(data)
    return data[:int(cutoff*len(data))], data[int(cutoff*len(data)):]

train, test = split_data(list(treebank.tagged_sents()), cutoff=0.9)
display(len(train), len(test))

In [ ]:
# 6.2 SOLUTION:


## Assignment 7: parsing (20 points)

In this assignment, you will perform evaluation over a constituency parser. The first bit of code is taken from the exercises and sets up a probabilistic context-free grammar and a Viterbi parser for it, over a sample from the Penn Treebank.

In [ ]:
from nltk.tree.tree import Tree
from nltk.corpus import treebank
from nltk.grammar import Nonterminal, induce_pcfg
from nltk.parse.viterbi import ViterbiParser

parsed_sents = treebank.parsed_sents()
grammar = induce_pcfg(Nonterminal("S"), [p for s in parsed_sents for p in s.productions()])
viterbi_parser = ViterbiParser(grammar)

The code below loads and displays two example test trees (for the sentences `They will join the board .` and `The student will do the work on Monday .`) containing a few parsing errors.

In [ ]:
with open('example_trees.txt', 'r') as in_file:
    example_trees = [Tree.fromstring(s.strip()) for s in in_file.readlines()]

display(example_trees[0])
display(example_trees[1])

The code below will compute the span representation for a given tree, as a first step in calculating parser evaluation metrics (precision, recall, F1).



In [ ]:
from typing import Tuple, List

def get_leaf_position_for_idx(tree, idx):
    return [i for i in range(len(tree.leaves())) if tree.leaf_treeposition(i) == idx][0]

def get_span_for_index(tree: Tree, idx: Tuple[int,...]):
    return [get_leaf_position_for_idx(tree, p) for p in tree.treepositions() 
            if p[:len(idx)] == idx and isinstance(tree[p], str)]

def get_internal_node_idxs(tree: Tree) -> List[Tuple[int,...]]:
    return [i for i in tree.treepositions() if isinstance(tree[i], Tree) and tree[i].height() > 1]
    
def compute_labeled_span_for_node(tree: Tree, idx: Tuple[int,...]) -> Tuple[str, int, int]:
    label = tree[idx].label()
    span = get_span_for_index(tree, idx)
    return label, span[0], span[-1]

def string_for_lspan(label: str, left: int, right: int):
    return f"{label}({left},{right})"

def get_labelled_spans_for_tree(tree: Tree) -> List[Tuple[str, int, int]]:
    # First grab the tree indices for internal nodes     
    internal_node_indices = get_internal_node_idxs(tree)
    # Compute the labelled spans for all internal nodes
    labelled_spans = [compute_labeled_span_for_node(tree, idx) for idx in internal_node_indices]
    return [string_for_lspan(label, left, right) for label, left, right in labelled_spans]

In [ ]:
example_spans1 = get_labelled_spans_for_tree(example_trees[0])
example_spans2 = get_labelled_spans_for_tree(example_trees[1])

display(example_spans1)
display(example_spans2)

### Part 7.1: Parser evaluation (10 points)

Do the following:
- Use the Viterbi parser from the previous exercise to extract the parses for these sentences.
- Finish the implementation for precision, recall and F1 score and calculate these metrics for the parser output against the two test sentences. Verify that you get the following scores:

| Sentence        | Precision   | Recall | F1 |
|:------------- |:-------------:|:------:|:--:|
| `They will join the board .`      | 0.82 | 0.82 | 0.82 |
| `The student will do the work on Monday .`    | 0.87  | 0.81 | 0.84 |

In [ ]:
# 7.1 SOLUTION:


### Part 7.1b (BONUS, 5 points):

This question can lead to bonus points, which may compensate any error you made elsewhere.

- What happens to the evaluation metrics if you switch the test tree and the parse tree around?
- What happens if you compare the first test tree against the second parsed sentence and vice versa?

In [ ]:
# 7.1b SOLUTION:


### Part 7.2: Large-Scale evaluation (10 points)

Now we must find out a way to evaluate on a larger test set of sentences. This set is given in the file `test_trees.txt`, which is already loaded by the code below. Do the following:

1. Parse each of the sentences.
2. Calculate the micro-averaged precision, recall and F1 scores. That is, you must first aggregate all predictions and then calculate the overall metrics! This time, the trees in the test set correspond to the *correct parses*.

*Note: running over all the test trees may take a while. Although it shouldn't take more than a few minutes at most, it is recommended to use `tqdm` when iterating over the test trees to keep track of the progress, as indicated in the code below.*

In [ ]:
# 7.2 SOLUTION:
from tqdm import tqdm

with open('test_trees.txt', 'r') as in_file:
    test_trees = [Tree.fromstring(s.strip()) for s in in_file.readlines()]

for tree in tqdm(test_trees, position=0):
    print("Your solution awaits!")